In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [4]:
# Debug the PortfolioSetup class and directly update the config
from config import PortfolioSetup
import config

# Let's debug what's happening
ps = PortfolioSetup()
print("PortfolioSetup instance:", ps)  
print("new_capital_growth_amt attribute:", ps.new_capital_growth_amt)
print("to_dict() result:", ps.to_dict())

# Force the value directly 
config.DEFAULT_PORTFOLIO_SETUP = {
    'initial_capital': 100000, 
    'initial_holdings': {}, 
    'new_capital_growth_pct': 0.0, 
    'new_capital_growth_amt': 0,  # Force the correct value
    'capital_growth_freq': 'M'
}
print("Manually updated DEFAULT_PORTFOLIO_SETUP:", config.DEFAULT_PORTFOLIO_SETUP)

# Also update DEFAULT_BACKTEST_PARAMS
config.DEFAULT_BACKTEST_PARAMS['setup'] = config.DEFAULT_PORTFOLIO_SETUP
print("Updated DEFAULT_BACKTEST_PARAMS setup:", config.DEFAULT_BACKTEST_PARAMS['setup'])


PortfolioSetup instance: PortfolioSetup(initial_capital=100000, initial_holdings={}, new_capital_growth_pct=0.0, new_capital_growth_amt=10000, capital_growth_freq='M')
new_capital_growth_amt attribute: 10000
to_dict() result: {'initial_capital': 100000, 'initial_holdings': {}, 'new_capital_growth_pct': 0.0, 'new_capital_growth_amt': 10000, 'capital_growth_freq': 'M'}
Manually updated DEFAULT_PORTFOLIO_SETUP: {'initial_capital': 100000, 'initial_holdings': {}, 'new_capital_growth_pct': 0.0, 'new_capital_growth_amt': 0, 'capital_growth_freq': 'M'}
Updated DEFAULT_BACKTEST_PARAMS setup: {'initial_capital': 100000, 'initial_holdings': {}, 'new_capital_growth_pct': 0.0, 'new_capital_growth_amt': 0, 'capital_growth_freq': 'M'}


In [5]:
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from config import DEFAULT_BACKTEST_PARAMS, DEFAULT_STRATEGY_CONFIG

scenario = Scenario(
    name=DEFAULT_BACKTEST_PARAMS["scenario_name"],
    start_date=DEFAULT_BACKTEST_PARAMS["start_date"],
    end_date=DEFAULT_BACKTEST_PARAMS["end_date"],
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    portfolio_setup=DEFAULT_BACKTEST_PARAMS["setup"],
    benchmark=DEFAULT_BACKTEST_PARAMS["benchmark"],
)

strategies = DEFAULT_STRATEGY_CONFIG.to_strategies()
scenario.set_strategies(strategies)

backtest = Backtest(scenario)
backtest.run(verbose=True)
backtest.generate_report(filename="test")

{'initial_capital': 100000, 'initial_holdings': {}, 'new_capital_growth_pct': 0.0, 'new_capital_growth_amt': 0, 'capital_growth_freq': 'M'}
Backtest starting... whomp whomp!
Total trading days: 855
Universe size: 480
Total strategies: 4
Starting in 2022-01-01
Ending in 2025-06-01


Backtesting by date x strategy:  97%|█████████▋| 826/855 [00:18<00:00, 37.85day/s]

Long trade amount 243 too large violates max trade constraint


Backtesting by date x strategy: 100%|██████████| 855/855 [00:18<00:00, 45.28day/s]


Ding ding ding! Backtest completed!
Report saved to: reports/test.pdf


'reports/test.pdf'

In [7]:
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from config import DEFAULT_BACKTEST_PARAMS, DEFAULT_STRATEGY_CONFIG

scenario = Scenario(
    name=DEFAULT_BACKTEST_PARAMS["scenario_name"],
    start_date=DEFAULT_BACKTEST_PARAMS["start_date"],
    end_date=DEFAULT_BACKTEST_PARAMS["end_date"],
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    portfolio_setup=DEFAULT_BACKTEST_PARAMS["setup"],
    benchmark=DEFAULT_BACKTEST_PARAMS["benchmark"],
)

strategies = DEFAULT_STRATEGY_CONFIG.to_strategies()
scenario.set_strategies(strategies)

backtest = Backtest(scenario)
backtest.run_batch(verbose=True)
backtest.generate_report(filename="test2")

{'initial_capital': 100000, 'initial_holdings': {}, 'new_capital_growth_pct': 0.0, 'new_capital_growth_amt': 0, 'capital_growth_freq': 'M'}
Backtest starting... swoosh!
Universe size: 480
Total trading days: 855
Total strategies: 4
Starting in 2022-01-01
Ending in 2025-06-01


Backtesting by strategy: 100%|██████████| 4/4 [00:03<00:00,  1.33strategy/s]


Long trade amount 243 too large violates max trade constraint
Backtest completed!
Report saved to: reports/test2.pdf


'reports/test2.pdf'

In [21]:
gs.run()

Running grid search with 5 parameter combinations...


Grid search progress:  60%|██████    | 3/5 [00:02<00:01,  1.62it/s]

Long trade amount 256 too large violates max trade constraint


Grid search progress: 100%|██████████| 5/5 [00:02<00:00,  2.16it/s]

Grid search completed! Found 5 valid results.


{'2': {'grid_num': 'grid_2',
  'param_name': 'triggers: rsi_x',
  'total_return': np.float64(21.22497519227921),
  'annualized_return': np.float64(21.783267808814205),
  'annualized_sharpe': np.float64(1.635308040683825),
  'annualized_ir': np.float64(-11.808057765325838),
  'average_holding_period': np.float64(240.28286852589642),
  'max_holding_amount': 371},
 '4': {'grid_num': 'grid_4',
  'param_name': 'triggers: rsi_x, z',
  'total_return': np.float64(60.124761708779),
  'annualized_return': np.float64(62.169423945721775),
  'annualized_sharpe': np.float64(1.4444610302384744),
  'annualized_ir': np.float64(-6.282609591365667),
  'average_holding_period': np.float64(228.75697211155378),
  'max_holding_amount': 350},
 '1': {'grid_num': 'grid_1',
  'param_name': 'triggers: macd_x',
  'total_return': np.float64(7.1112181840451125),
  'annualized_return': np.float64(7.248192199601906),
  'annualized_sharpe': np.float64(2.0250582831107575),
  'annualized_ir': np.float64(-2.01290751745636

In [19]:
gs.results_to_dataframe()

,grid_num,param_name,total_return,annualized_return,annualized_sharpe,annualized_ir,average_holding_period,max_holding_amount
0,1,triggers: rsi_x,21.224975,21.783268,1.635308,-11.808058,240.282869,371
1,3,"triggers: rsi_x, z",60.124762,62.169424,1.444461,-6.282610,228.756972,350
2,0,triggers: macd_x,7.111218,7.248192,2.025058,-2.012908,230.334661,453
3,2,triggers: macd_x | filter: b_bands,7.607369,7.833522,1.361947,-7.495404,249.876000,436
4,4,"triggers: macd_x, b_bands, z, rsi_x",55.883907,57.752904,1.329464,-5.047790,216.653386,310


In [15]:
gs.get_grid_search_schedule()

,grid_num,param_name
0,1,triggers: rsi_x
1,3,"triggers: rsi_x, z"
2,0,triggers: macd_x
3,4,"triggers: macd_x, b_bands, z, rsi_x"


In [16]:
gs.grid_params

[{'triggers': [<Strategies.MACD_CROSSOVER: 'macd_x'>], 'filter': []},
 {'triggers': [<Strategies.RSI_CROSSOVER: 'rsi_x'>], 'filter': []},
 {'triggers': [<Strategies.MACD_CROSSOVER: 'macd_x'>],
  'filter': [<Strategies.BOLLINGER_BANDS: 'b_bands'>]},
 {'triggers': [<Strategies.RSI_CROSSOVER: 'rsi_x'>,
   <Strategies.Z_SCORE_MEAN_REVERSION: 'z'>],
  'filter': []},
 {'triggers': [<Strategies.MACD_CROSSOVER: 'macd_x'>,
   <Strategies.BOLLINGER_BANDS: 'b_bands'>,
   <Strategies.Z_SCORE_MEAN_REVERSION: 'z'>,
   <Strategies.RSI_CROSSOVER: 'rsi_x'>],
  'filter': []}]